In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import calibration_curve, CalibrationDisplay
from sklearn.metrics import roc_curve, auc
from matplotlib.gridspec import GridSpec
import matplotlib.ticker as ticker

In [6]:
import pandas as pd

# Read in data and assign X and y
data = pd.read_csv('../data/Milestone2_data/Q2_train_data.csv', index_col=0)
print(data.columns)
print(data[['distance', 'angle' ]])
print(data.head())
print(data.tail())

Index(['gameID', 'distance', 'angle', 'isGoal', 'emptyNet'], dtype='object')
          distance    angle
0        13.000000  22.6199
1        13.341664  77.0054
2       118.270876 -18.7413
3        57.974132 -14.9951
4        61.717096  26.9802
...            ...      ...
327249   38.418745 -38.6598
327250    8.544004  20.5560
327251   55.326305  -6.2258
327252   58.855756   9.7824
327253   12.000000   0.0000

[327254 rows x 2 columns]
       gameID    distance    angle  isGoal  emptyNet
0  2016020001   13.000000  22.6199       0         0
1  2016020001   13.341664  77.0054       0         0
2  2016020001  118.270876 -18.7413       0         0
3  2016020001   57.974132 -14.9951       0         0
4  2016020001   61.717096  26.9802       0         0
            gameID   distance    angle  isGoal  emptyNet
327249  2019030416  38.418745 -38.6598       0         0
327250  2019030416   8.544004  20.5560       0         0
327251  2019030416  55.326305  -6.2258       0         0
327252  201903

In [7]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [5]:
# # Initialize WandB
# wandb.init(
#     project="IFT6758.2024-A04",  
#     job_type="logistic-regression-angle-and_distance",
#     name="Logistic Regression (Angle & Distance)")

In [13]:
#Train Model

import wandb
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, log_loss
import joblib

# Initialize WandB
wandb.init(
    project="IFT6758.2024-A04",  
    job_type="expected-goal-model-angle-and-distance",
    name="expected-goal-model-angle-and-distance",
     config={
        "test_size": 0.2,          # Fraction of data used for testing
        "random_state": 42,        # Random seed for reproducibility
        "learning_rate": 0.001,    # Example hyperparameter
    })

# Configuration (hyperparameters)
config = wandb.config

# Split the data into features and labels
X = data[["distance", "angle"]]
y = data["isGoal"]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=config.test_size, random_state=config.random_state)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)
logloss = log_loss(y_test, y_proba)

# Log metrics to WandB
wandb.log({"accuracy": accuracy, "roc_auc": roc_auc, "F1-Score": f1, "Log Loss": logloss})

# Save the model locally
model_path = "expected-goal-model-angle-and-distance.pkl"
joblib.dump(model, model_path)

# Log the model as an artifact
artifact = wandb.Artifact("expected-goal-model-angle-and-distance", type="model")
artifact.add_file(model_path)
wandb.log_artifact(artifact)

# Finish the WandB run
wandb.finish()

print(f"Model trained and logged to WandB with accuracy: {accuracy:.2f}, ROC AUC: {roc_auc:.2f}")


F1-Score,▁
Log Loss,▁
accuracy,▁
roc_auc,▁
F1-Score,0
Log Loss,0.30123
accuracy,0.9035
roc_auc,0.69601


Model trained and logged to WandB with accuracy: 0.90, ROC AUC: 0.70


In [12]:
#Train Model

# Initialize WandB
wandb.init(
    project="IFT6758.2024-A04",  
    job_type="expected-goal-distance",
    name="expected-goal-distance",
     config={
        "test_size": 0.2,          # Fraction of data used for testing
        "random_state": 42,        # Random seed for reproducibility
        "learning_rate": 0.001,    # Example hyperparameter
    })

# Configuration (hyperparameters)
config = wandb.config

# Split the data into features and labels
X = data[["distance"]]
y = data["isGoal"]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=config.test_size, random_state=config.random_state)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)
logloss = log_loss(y_test, y_proba)

# Log metrics to WandB
wandb.log({"accuracy": accuracy, "roc_auc": roc_auc, "F1-Score": f1, "Log Loss": logloss})

# Save the model locally
model_path = "expected-goal-distance.pkl"
joblib.dump(model, model_path)

# Log the model as an artifact
artifact = wandb.Artifact("expected-goal-distance", type="model")
artifact.add_file(model_path)
wandb.log_artifact(artifact)

# Finish the WandB run
wandb.finish()

print(f"Model trained and logged to WandB with accuracy: {accuracy:.2f}, ROC AUC: {roc_auc:.2f}")


F1-Score,▁
Log Loss,▁
accuracy,▁
roc_auc,▁
F1-Score,0
Log Loss,0.30128
accuracy,0.9035
roc_auc,0.69578


Model trained and logged to WandB with accuracy: 0.90, ROC AUC: 0.70


In [16]:
import pandas as pd
import joblib

# Load the saved model
model_path = "logistic_regression_distance.pkl"
model = joblib.load(model_path)

# Example input data for prediction
new_data = pd.DataFrame({"distance": [10.5, 15.2, 20.1]})  # Replace with your own data

# Make predictions
predictions = model.predict(new_data)
probabilities = model.predict_proba(new_data)[:, 1]  # Probability of being a goal (class 1)

# Display predictions and probabilities
for i, dist in enumerate(new_data["distance"]):
    print(f"Distance: {dist}, Predicted: {predictions[i]}, Probability of Goal: {probabilities[i]:.2f}")


Distance: 10.5, Predicted: 0, Probability of Goal: 0.16
Distance: 15.2, Predicted: 0, Probability of Goal: 0.14
Distance: 20.1, Predicted: 0, Probability of Goal: 0.13


In [15]:
import wandb

# Login (optional if already logged in)
wandb.login()

# Project and artifact details
project_name = "IFT6758.2024-A04"
artifact_path = "thalia-cantero-udem/IFT6758.2024-A04/expected-goal-distance:v1"

# Initialize wandb for the project
wandb.init(project=project_name)

# Fetch the artifact
artifact = wandb.use_artifact(artifact_path)
artifact_dir = artifact.download()

print(f"Artifact downloaded to: {artifact_dir}")



wandb:   1 of 1 files downloaded.  


Artifact downloaded to: /Users/thaliacantero/Documents/UdeM/2024 Automne/DataScience/Project/project-template/source/artifacts/expected-goal-distance:v1


In [14]:
import requests

# URL du point de terminaison Flask
url = "http://localhost:5000/download_registry_model"

# Payload JSON avec les données nécessaires
#expected-goal-distance:v1
#expected-goal-model-angle-and-distance:v1
payload = {
    "entity":"thalia-cantero-udem",
    "project":"IFT6758.2024-A04",
    "model": "expected-goal-distance",
    "version": "v1"
    }
#runs/
# Envoi de la requête POST
response = requests.post(url, json=payload)

# Affichage de la réponse
print(response.status_code)  # Devrait être 200 si tout est correct
print(response.json())       # Réponse JSON renvoyée par Flask


403


JSONDecodeError: Expecting value: line 1 column 1 (char 0)